In [ ]:
!pip install bdsf
!pip install pyregion

In [ ]:
import numpy as np
from astropy.io import fits
from astropy.utils.data import get_pkg_data_filename
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import bdsf as sf
import pyregion
from astropy.wcs import WCS

In [ ]:
# image to be used for source identification - cutout from 144MHz COSMOS (PI: Vardoulaki)
# let's see what we are working with 

# Load the image data from the local file
image_file = get_pkg_data_filename('cosmos144MHz_zoom_2naxis.fits')
image_data = fits.getdata(image_file, ext=0)
image_header = fits.getheader(image_file, ext=0)


plt.figure()
plt.imshow(image_data,vmin=1e-5, vmax=1e-3,origin='lower',interpolation='nearest', aspect='auto',cmap='inferno')
plt.title(r'COSMOS 144 MHz')
plt.xlabel('Right Ascension (J2000)')
plt.ylabel('Declination (J2000)')

In [ ]:
#run pybdsf code with default settings
sf.process_image("./cosmos144MHz_zoom.fits", thresh_isl=4.0, thresh_pix=5.0, rms_box=(150, 15), 
                 rms_map=True, mean_map='zero', ini_method='intensity', adaptive_rms_box=True, 
                 adaptive_thresh=150, rms_box_bright=(60, 15), group_by_isl=False, 
                 group_tol=10.0, output_opts=True, output_all=True, atrous_do=True,atrous_jmax=4, 
                 flagging_opts=True, flag_maxsize_fwhm=0.5,advanced_opts=True, blank_limit=None)

PyBDSF creates a folder with all results of the fitting, and the final catalogues plus locations for the identified radio islands. 

cosmos144MHz_zoom_pybdsf: 

background
catalogues
misc
model
residual
wavelet

In catalogues, we can find the locations of the islands and mark them on the radio image. We also find the source catalogue. For more info on PyBDSF https://pybdsf.readthedocs.io/en/latest/

In [ ]:
# plot the radio map
image_file2 = fits.open('cosmos144MHz_zoom_2naxis.fits', memmap=True) 
image_data2 = image_file2[0].data
image_header2 = image_file2[0].header

wcs = WCS(image_header,naxis=2)
print('---------',wcs)

plt.figure()

fig, ax = plt.subplots(figsize=(8,6))
ax = plt.subplot(projection=wcs)
    
plt.imshow(image_data,vmin=1e-5, vmax=1e-3,origin='lower',interpolation='nearest', aspect='auto',cmap='gray')

plt.title(r'COSMOS 144 MHz')
plt.xlabel('Right Ascension (J2000)')
plt.ylabel('Declination (J2000)')


# get the regions from the file
region_name = "cosmos144MHz_zoom_pybdsf/catalogues/cosmos144MHz_zoom.pybdsf.srl.ds9.reg"

r = pyregion.open(region_name).as_imagecoord(header=image_header2)

patch_list, artist_list = r.get_mpl_patches_texts()


# ax is a mpl Axes object
for p in patch_list:
    ax.add_patch(p)
# for t in artist_list:
#     ax.add_artist(t)

# Save the figure to a PNG file
output_file = 'cosmos144MHz_zoom.png'
plt.savefig(output_file, dpi=300, bbox_inches='tight')

Let's see what is in the catalogue

In [ ]:
pybdsf_cat = fits.open('cosmos144MHz_zoom_pybdsf/catalogues/cosmos144MHz_zoom.pybdsf.srl.FITS')
pybdsf_cat.info()

In [ ]:
print(pybdsf_cat[1].columns)

In [ ]:
pybdsf_data = pybdsf_cat[1].data

# make histogram of flux density of islands identified

# Create logarithmically spaced bins
min_flux = np.nanmin(pybdsf_data['Total_flux'])
max_flux = np.nanmax(pybdsf_data['Total_flux'])
bins = np.logspace(np.log10(min_flux), np.log10(max_flux), num=50)  # Adjust num for the number of bins you want

# Create the histogram
plt.figure()
flux_hist = plt.hist(pybdsf_data['Total_flux'], bins=bins, edgecolor='black', color='green')

# Set the x-axis to logarithmic scale
plt.xscale('log')

# Set the labels and title
plt.xlabel('Flux density (Jy)', fontsize=20)
plt.ylabel('Number of Sources', fontsize=20)
plt.title('Histogram of Flux Density', fontsize=20)